In [1]:
cd ..

/Users/kimyoungh/GoogleDrive/rich/main/rich


In [10]:
import numpy as np
import pandas as pd

import torch

from data_processing.price_factor_processor import PriceFactorProcessor
from data_processing.processing_beamtrader_training_data import ProcessingBeamTraderTrainingData

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
global_index = pd.read_parquet(
    'trading_data/global_index_series.pq')

In [4]:
global_index.index.name = None
global_index.columns.name = None

gindex = pd.Series(global_index.index.to_pydatetime())
gindex = gindex.apply(lambda x: x.strftime('%Y%m%d'))

global_index.index = gindex

global_returns = global_index.pct_change().iloc[1:]

In [5]:
returns = pd.read_csv('index_strategy_returns.csv',
                    index_col=0, header=0)
returns.index = returns.index.astype(str)

In [6]:
rindex = returns.loc[global_returns.index[0]:].index

global_returns = global_returns.reindex(rindex).ffill()
returns = returns.reindex(rindex)

In [7]:
returns

,K200,KQ,K200_i,KQ_i
20060103,0.006152,0.007250,-0.006152,-0.007250
20060104,-0.006059,-0.005159,0.006059,0.005159
20060105,0.011464,0.014172,-0.011464,-0.014172
20060106,-0.003151,0.009075,0.003151,-0.009075
20060109,-0.007321,-0.012164,0.007321,0.012164
...,...,...,...,...
20220401,0.004818,0.007176,-0.004818,-0.007176
20220404,-0.000548,0.005436,0.000548,-0.005436
20220405,-0.010226,-0.009806,0.010226,0.009806
20220406,-0.014626,-0.016095,0.014626,0.016095


In [8]:
pbtt = ProcessingBeamTraderTrainingData(
    returns, seq_len=10, trading_fee=0.004,
    beam_width=100, n_expand=200, discount=0.99,
)

In [9]:
factors, best_seqs, best_rebal_seqs,\
    best_rews, best_rebal_rews,\
    best_vals, best_rebal_vals,\
    trade_dates, factors_index, regimes = pbtt.generate_target_data()

In [11]:
print(factors.shape)
print(best_seqs.shape)
print(best_rebal_seqs.shape)
print(trade_dates.shape)
print(factors_index.shape)
print(regimes.shape)

(3759, 4, 69)
(3759, 10)
(3759, 5, 10)
(3759,)
(3759, 4, 2)
(3759,)


In [12]:
pfp = PriceFactorProcessor(global_returns)
gfactors, gfactors_index, gtrade_dates = pfp.calculate_mf()

In [15]:
gfactors = gfactors[:len(factors)]
gfactors_index = gfactors_index[:len(factors)]
gtrade_dates = gtrade_dates[:len(factors)]

In [17]:
print(gfactors.shape)
print(gfactors_index.shape)
print(gtrade_dates.shape)

(3759, 173, 69)
(3759, 173, 2)
(3759,)


In [18]:
factors_train = factors[:-500]
factors_test = factors[-500:]

gfactors_train = gfactors[:-500]
gfactors_test = gfactors[-500:]

best_seqs_train = best_seqs[:-500]
best_seqs_test = best_seqs[-500:]

best_rebal_seqs_train = best_rebal_seqs[:-500]
best_rebal_seqs_test = best_rebal_seqs[-500:]

best_rews_train = best_rews[:-500]
best_rews_test = best_rews[-500:]

best_rebal_rews_train = best_rebal_rews[:-500]
best_rebal_rews_test = best_rebal_rews[-500:]

best_vals_train = best_vals[:-500]
best_vals_test = best_vals[-500:]

best_rebal_vals_train = best_rebal_vals[:-500]
best_rebal_vals_test = best_rebal_vals[-500:]

trade_dates_train = trade_dates[:-500]
trade_dates_test = trade_dates[-500:]

factors_index_train = factors_index[:-500]
factors_index_test = factors_index[-500:]

gfactors_index_train = gfactors_index[:-500]
gfactors_index_test = gfactors_index[-500:]

regimes_train = regimes[:-500]
regimes_test = regimes[-500:]

In [19]:
data_path = "./train_data/beam_trader/"

np.save(data_path+"factors_train.npy", factors_train,
        allow_pickle=True)
np.save(data_path+"factors_test.npy", factors_test,
        allow_pickle=True)

np.save(data_path+"gfactors_train.npy", gfactors_train,
        allow_pickle=True)
np.save(data_path+"gfactors_test.npy", gfactors_test,
        allow_pickle=True)

np.save(data_path+"best_seqs_train.npy", best_seqs_train,
        allow_pickle=True)
np.save(data_path+"best_seqs_test.npy", best_seqs_test,
        allow_pickle=True)

np.save(data_path+"best_rebal_seqs_train.npy",
        best_rebal_seqs_train,
        allow_pickle=True)
np.save(data_path+"best_rebal_seqs_test.npy",
        best_rebal_seqs_test,
        allow_pickle=True)

np.save(data_path+"best_rews_train.npy", best_rews_train,
        allow_pickle=True)
np.save(data_path+"best_rews_test.npy", best_rews_test,
        allow_pickle=True)

np.save(data_path+"best_rebal_rews_train.npy",
        best_rebal_rews_train,
        allow_pickle=True)
np.save(data_path+"best_rebal_rews_test.npy",
        best_rebal_rews_test,
        allow_pickle=True)

np.save(data_path+"best_vals_train.npy", best_vals_train,
        allow_pickle=True)
np.save(data_path+"best_vals_test.npy", best_vals_test,
        allow_pickle=True)

np.save(data_path+"best_rebal_vals_train.npy",
        best_rebal_vals_train,
        allow_pickle=True)
np.save(data_path+"best_rebal_vals_test.npy",
        best_rebal_vals_test,
        allow_pickle=True)

np.save(data_path+"trade_dates_train.npy",
        trade_dates_train,
        allow_pickle=True)
np.save(data_path+"trade_dates_test.npy",
        trade_dates_test,
        allow_pickle=True)

np.save(data_path+"factors_index_train.npy",
        factors_index_train,
        allow_pickle=True)
np.save(data_path+"factors_index_test.npy",
        factors_index_test,
        allow_pickle=True)

np.save(data_path+"gfactors_index_train.npy",
        gfactors_index_train,
        allow_pickle=True)
np.save(data_path+"gfactors_index_test.npy",
        gfactors_index_test,
        allow_pickle=True)

np.save(data_path+"regimes_train.npy", regimes_train,
        allow_pickle=True)
np.save(data_path+"regimes_test.npy", regimes_test,
        allow_pickle=True)

In [20]:
best_rebal_seqs[700]

array([[0, 4, 4, 1, 3, 1, 1, 1, 1, 3],
       [1, 4, 4, 1, 3, 1, 1, 1, 1, 3],
       [2, 4, 4, 1, 3, 1, 1, 1, 1, 3],
       [3, 3, 3, 1, 3, 1, 1, 1, 1, 3],
       [4, 4, 4, 1, 3, 1, 1, 1, 1, 3]])